In [1]:
param_date = '20240621'

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [3]:
input_ = pd.read_excel(f'./{param_date}/4.part.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_.drop_duplicates(['Info'], keep='first', ignore_index=True)

print(f'总数量：{len(input_)}')
input_['Info'].tolist()

总数量：1


['https://www.rockauto.com/en/moreinfo.php?pk=11158045&cc=0&pt=5756&jsn=3']

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

In [6]:
a = 0

resp = requests.get(input_.loc[a, 'Info'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<!DOCTYPE HTML>\n<html class="no-js REPLACEHTMLCLASS" onclick="" lang="en">\n<head>\n<meta name="robots" content="all">\n<meta http-equiv="Content-Security-Policy" content="script-src \'self\' \'unsafe-inline\' \'unsafe-eval\' *.rockauto.com; base-uri \'self\' *.rockauto.com; ">\n<link rel="stylesheet" type="text/css" href="/jslib/combined/css/desktopmoreinfo.css?1719048256">\n\n<script>\nif(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};\n_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift()

In [7]:
from bs4 import BeautifulSoup
from lxml import etree

In [8]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./info.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE HTML>
<html class="no-js REPLACEHTMLCLASS" lang="en" onclick="">
 <head>
  <meta content="all" name="robots"/>
  <meta content="script-src 'self' 'unsafe-inline' 'unsafe-eval' *.rockauto.com; base-uri 'self' *.rockauto.com; " http-equiv="Content-Security-Policy"/>
  <link href="/jslib/combined/css/desktopmoreinfo.css?1719048256" rel="stylesheet" type="text/css"/>
  <script>
   if(!window.RABoot){var no_js="no-js",has_js="has-js",c=document.documentElement.className.replace(no_js,has_js);-1==c.indexOf(has_js)&&(c+=" "+has_js);document.documentElement.className=c;RABoot=function(){var f=function(){};f.prototype={};f.WaitFor=function(a,b){if(_isset(b)&&"function"===typeof b){_isarray(a)||(a=[a]);var d={s:a,w:b,a:{}};CheckReady(d)?d.w():Waiting.push(d)}};_isset=function(a){return"undefined"!=typeof a&&null!=a};_isin=function(a,b){return a in b};_isarray=function(a){return Array.isArray(a)};
_getref=function(a){var b=null;a=a.split(".");if(0<a.length){"window"==a[0]&&a.shift();for

In [9]:
html.xpath('//span[@class="la-btn-v"]/text()')[0].strip() != 'Continue Shopping'

False

In [10]:
list_tr = html.xpath('//table[@class="moreinfotable"]/tr')

list_tr

[<Element tr at 0x10fc07ac0>,
 <Element tr at 0x10fc24600>,
 <Element tr at 0x10fc24640>]

In [11]:
import json

In [12]:
dict_specification = {}
for i in range(len(list_tr)):
    if i == 0:
        continue

    list_td = [td.strip() for td in list_tr[i].xpath('./td/text()')]
    dict_specification[str(i)] = {list_td[0]: list_td[1]}

json_specification = json.dumps(dict_specification)

json_specification

'{"1": {"Sprockets Included": "No"}, "2": {"Timing Chain Type": "Silent"}}'

In [13]:
list_tr = html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-1") or (contains(@class, "altrow-b-0")))]/tr')

list_tr

[<Element tr at 0x10feacb00>,
 <Element tr at 0x10fc25600>,
 <Element tr at 0x10fc25d40>,
 <Element tr at 0x10fc25dc0>,
 <Element tr at 0x10fc25e00>,
 <Element tr at 0x10fc25e80>,
 <Element tr at 0x10fc25ec0>]

In [14]:
dict_kit = {}
for i in range(len(list_tr)):
    dict_kit[str(i)] = {}

    list_td = list_tr[i].xpath('./td')
    dict_kit[str(i)]['quantity'] = list_td[0].xpath('./text()')[0].strip()
    dict_kit[str(i)]['component_tag'] = list_td[1].xpath('./text()')[0].strip()

    json_ = json.loads(list_td[2].xpath('./@value')[0])
    dict_kit[str(i)]['parttype'] = json_['parttype'].strip()
    dict_kit[str(i)]['partkey'] = json_['partkey'].strip()

    json_ = json.loads(list_tr[i].xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
    dict_kit[str(i)]['partnumber'] = json_['partnumber'].strip()
    dict_kit[str(i)]['catalogname'] = json_['catalogname'].strip()

    dict_kit[str(i)]['url'] = f'''https://www.rockauto.com/en/parts/{dict_kit[str(i)]['partkey']},{dict_kit[str(i)]['partnumber']},{dict_kit[str(i)]['component_tag']},{dict_kit[str(i)]['parttype']}'''

    a_ = list_tr[i].xpath('./div[@class="listing-text-row-moreinfo-truck"]/a/@href')
    dict_kit[str(i)]['info'] =  f'''https://www.rockauto.com/en/moreinfo.php?pk={dict_kit[str(i)]['catalogname']}&pt={dict_kit[str(i)]['parttype']}&Lennon=1''' if len(a_) == 0 else a_[0].strip()

    list_src = [] if len(list_td[3].xpath('./div/input/@value')) == 0 else json.loads(list_td[3].xpath('./div/input/@value')[0])['Slots']
    dict_src = {}
    for j in range(len(list_src)):
        dict_src[str(j)] = 'https://www.rockauto.com' + list_src[j]['ImageData']['Full'].strip()
    dict_kit[str(i)]['json_src'] = json.dumps(dict_src)

json_kit = json.dumps(dict_kit)

json_kit

'{"0": {"quantity": "1", "component_tag": "Timing Chain", "parttype": "5724", "partkey": "8306448", "partnumber": "715F", "catalogname": "MELLING", "url": "https://www.rockauto.com/en/parts/8306448,715F,Timing Chain,5724", "info": "https://www.rockauto.com/en/moreinfo.php?pk=8306448&cc=0&pt=5724", "json_src": "{\\"0\\": \\"https://www.rockauto.com/info/583/715F-1-ANG.jpg\\"}"}, "1": {"quantity": "1", "component_tag": "Timing Chain Guide / Damper", "parttype": "5728", "partkey": "8307424", "partnumber": "BG5515", "catalogname": "MELLING", "url": "https://www.rockauto.com/en/parts/8307424,BG5515,Timing Chain Guide / Damper,5728", "info": "https://www.rockauto.com/en/moreinfo.php?pk=8307424&cc=0&pt=5728", "json_src": "{\\"0\\": \\"https://www.rockauto.com/info/583/BG5515-1-SID.jpg\\"}"}, "2": {"quantity": "1", "component_tag": "Timing Chain Tensioner", "parttype": "5736", "partkey": "8308132", "partnumber": "BT5514", "catalogname": "MELLING", "url": "https://www.rockauto.com/en/parts/8308

In [15]:
df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info'],
                         'Json_Specification': json_specification,
                         'Json_Kit': json_kit}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Info,Json_Specification,Json_Kit
0,https://www.rockauto.com/en/moreinfo.php?pk=11...,"{""1"": {""Sprockets Included"": ""No""}, ""2"": {""Tim...","{""0"": {""quantity"": ""1"", ""component_tag"": ""Timi..."


In [16]:
crawler_status = 'ok'

crawler_status

'ok'

In [17]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [18]:
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['Info'], ascending=[True], ignore_index=True)
    output_correct.to_excel('./test_info.xlsx', index=False)

output_correct

,Info,Json_Specification,Json_Kit
0,https://www.rockauto.com/en/moreinfo.php?pk=11...,"{""1"": {""Sprockets Included"": ""No""}, ""2"": {""Tim...","{""0"": {""quantity"": ""1"", ""component_tag"": ""Timi..."


In [19]:
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['Info'], ascending=[True], ignore_index=True)
    output_error.to_excel('./test_info_error.xlsx', index=False)

output_error

""
